In [ ]:
Add latex macros$$\newcommand{\V}[1]{{\boldsymbol{#1}}}\newcommand{mean}[1]{{\mathbb{E}\left[#1\right]}}\newcommand{var}[1]{{\mathbb{V}\left[#1\right]}}\newcommand{covar}[2]{\mathbb{C}\text{ov}\left[#1,#2\right]}\newcommand{corr}[2]{\mathbb{C}\text{or}\left[#1,#2\right]}\newcommand{argmin}{\mathrm{argmin}}\def\rv{z}\def\reals{\mathbb{R}}\def\pdf{\rho}\def\rvdom{\Gamma}\def\coloneqq{\colon=}\newcommand{norm}{\lVert #1 \rVert}\def\argmax{\operatorname{argmax}}\def\ai{\alpha}\def\bi{\beta}\newcommand{\dx}[1]{\;\mathrm{d}#1}$$


# End-to-End Model Analysis
This tutorial describes how to use each of the major model analyses in Pyapprox
following the exposition in [PYAPPROX2022]_.

First lets load all the necessary modules and set the random seeds for reproducibility.


In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import torch
import time
import warnings
from pyapprox.util.configure_plots import mathrm_label, mathrm_labels
from pyapprox.variables import (
    IndependentMarginalsVariable, print_statistics, AffineTransform)
from pyapprox.benchmarks import setup_benchmark, list_benchmarks
from __util import pyapprox_fun_1, pyapprox_fun_2
from pyapprox.interface.wrappers import (
    ModelEnsemble, TimerModel, WorkTrackingModel,
    evaluate_1darray_function_on_2d_array)
from pyapprox.surrogates import adaptive_approximate
from pyapprox.analysis.sensitivity_analysis import (
    run_sensitivity_analysis, plot_sensitivity_indices)
from pyapprox.bayes.markov_chain_monte_carlo import (
    loglike_from_negloglike, MCMCVariable)
from pyapprox.expdesign.bayesian_oed import get_bayesian_oed_optimizer
from pyapprox import multifidelity
warnings.filterwarnings("ignore", category=DeprecationWarning)
np.random.seed(2)
torch.manual_seed(2)

The tutorial can save the figures to file if desired. If you do want the plots
set savefig=True



In [ ]:
savefig = False

The following code shows how to create and sample from two independent uniform random variables defined on [−2, 2]. We use uniform variables here, but any marginal from the scipy.stats module can be used.



In [ ]:
nsamples = 30
univariate_variables = [stats.uniform(-2, 4), stats.uniform(-2, 4)]
variable = IndependentMarginalsVariable(univariate_variables)
samples = variable.rvs(nsamples)
print_statistics(samples)

PyApprox supports various types of variable transformations. The following code
shows how to use an affinte transformation to map samples from variables
to samples from the variable's canonical form.



In [ ]:
var_trans = AffineTransform(variable)
canonical_samples = var_trans.map_to_canonical(samples)
print_statistics(canonical_samples)

Pyapprox provides many utilities for interfacing with complex numerical codes
Below we take two functions, see `sphx_glr_auto_examples_plot_interface.py`
for thier definitions and show how to use the ModelEnsemble and
WorkTrackingModel to evaluate two models at once and time the wall time
of each evaluation of each model. The last print statement
prints the median execution time of each model



In [ ]:
model_ensemble = ModelEnsemble([pyapprox_fun_1, pyapprox_fun_2])
timer_fun_ensemble = TimerModel(model_ensemble)
worktracking_fun_ensemble = WorkTrackingModel(
    timer_fun_ensemble, num_config_vars=1)
fun_ids = np.ones(nsamples)
fun_ids[:nsamples//2] = 0
ensemble_samples = np.vstack([samples, fun_ids])
values = worktracking_fun_ensemble(ensemble_samples)
query_fun_ids = np.atleast_2d([0, 1])
print(worktracking_fun_ensemble.work_tracker(query_fun_ids))

Pyapprox provide numerous benchmarks for verifying, validating and comparing
model analysis algorithms. The following list the names of all benchmarks and
then creates a benchmark that can be used to test the creation of surrogates,
Bayesian inference, and optimal experimental design. This benchmark requires
determining the true coefficients of the Karhunene Loeve expansion (KLE)
used to characterize the uncertain diffusivity field of an advection
diffusion equation. See documentation of the benchmark for more details).



In [ ]:
print(list_benchmarks())
noise_stdev = 1e-1
inv_benchmark = setup_benchmark(
    "advection_diffusion_kle_inversion", kle_nvars=3,
    noise_stdev=noise_stdev, nobs=10, kle_length_scale=0.5)
print(inv_benchmark.keys())

The following plots the modes of the KLE



In [ ]:
fig, axs = plt.subplots(
    1, inv_benchmark.KLE.nterms, figsize=(8*inv_benchmark.KLE.nterms, 6))
for ii in range(inv_benchmark.KLE.nterms):
    inv_benchmark.mesh.plot(inv_benchmark.KLE.eig_vecs[:, ii:ii+1], 50,
                            ax=axs[ii])

PyApprox provides many popular methods for constructing surrogates
that once constructed can be evaluated in place of a computaionally
expensive simulation model in model analyses. The following code creates
a Gaussian process (GP) surrogate. The function used to construct the surrogate
takes a callback which is evaluated each time the adaptive surrogate is refined.
Here we use to compute the error of the surrogate as it is constructed using
validation data. Uncomment the code to use a polynomial based surrogate instead
of a GP. The user does not have to change any subsequent code



In [ ]:
validation_samples = inv_benchmark.variable.rvs(100)
validation_values = inv_benchmark.negloglike(validation_samples)
nsamples, errors = [], []
def callback(approx):
    nsamples.append(approx.num_training_samples())
    error = np.linalg.norm(
        approx(validation_samples)-validation_values, axis=0)
    error /= np.linalg.norm(validation_values, axis=0)
    errors.append(error)

approx_result = adaptive_approximate(
    inv_benchmark.negloglike, inv_benchmark.variable, "gaussian_process",
    {"max_nsamples": 30, "ncandidate_samples": 2e3, "verbose": 0,
     "callback": callback, "kernel_variance": 400})

# approx_result = adaptive_approximate(
#     inv_benchmark.negloglike, inv_benchmark.variable, "polynomial_chaos",
#     {"method": "leja", "options": {
#         "max_nsamples": 100, "ncandidate_samples": 3e3, "verbose": 0,
#         "callback": callback}})
approx = approx_result.approx

We can plot the errors obtained from the callback with



In [ ]:
ax = plt.subplots(figsize=(8, 6))[1]
ax.loglog(nsamples, errors, "o-")
ax.set_xlabel(mathrm_label("No. Samples"))
ax.set_ylabel(mathrm_label("Error"))
if savefig:
    plt.savefig("gp-error-plot.pdf")

Now we will perform a sensitivity analysis. Specifically we compute
variance based sensitivity indices that measure the impact of each KLE mode
on the mismatch between the observed data and the model predictions.
We use the negative log likelihood to characterize this mismatch.
Here we have used the surrogate to speed up the computation of the sensitivity
indices. Uncomment the commented code to use the numerical model. Note
the drastic increase in computational cost. Warning: using the numerical model
will take many minutes. The plots in the figure, generated from
left to right are: main effect, largest Sobol indices and total effect indices.



In [ ]:
sa_result = run_sensitivity_analysis(
    "surrogate_sobol", approx, inv_benchmark.variable)
# sa_result = run_sensitivity_analysis(
#     "sobol", benchmark.negloglike, inv_benchmark.variable)
axs = plot_sensitivity_indices(
    sa_result)[1]
if savefig:
    plt.savefig("gp-sa-indices.pdf", bbox_inches="tight")

Now we will use the surrogate with Bayesian inference to learn the
coefficients of the KL. Specifically we will draw a set of samples from
the posterior distribution of the KLE given the observed data provided
in the benchmark.

But First we will improve the accuracy of the surrogate
and print out the error which can be compared to the errors previously plotted.
The error of the original surrogate was kept low to demonstrate the ability
to quantify error in the sensitivity indices from using a surrogate.



In [ ]:
approx.refine(100)
error = np.linalg.norm(
    approx(validation_samples)-validation_values, axis=0)
error /= np.linalg.norm(validation_values, axis=0)
print("Surrogate", error)

Now create a MCMCVariable to sample from the posterior. The benchmark
has already formulated the negative log likelihood that is needed. Here
we will use the NUTS sampler from PyMC3. This can run many MCMC chains at once
by setting njobs > 1. However using njobs>1 cannot be used unless the srcipt
is invoked inside if __name__ == __main__: and so njobs>1
is not used for this tutorial.
Uncomment the commented code to use the numerical model instead of the surrogate
with the MCMC algorithm. Again note the significant increase in computational
time



In [ ]:
algorithm, npost_samples, njobs = "nuts", 100, 1
# loglike = partial(loglike_from_negloglike, inv_benchmark.negloglike)
loglike = partial(loglike_from_negloglike, approx)
mcmc_variable = MCMCVariable(
    inv_benchmark.variable, loglike, algorithm, njobs=njobs, loglike_grad=True)
print(mcmc_variable)
post_samples = mcmc_variable.rvs(npost_samples)
map_sample = mcmc_variable.maximum_aposteriori_point()

Now plot the posterior samples with the 2D Marginals of the posterior. Note
do not do this with the numerical model as this would take an eternity due
to the cost of evaluating the numerical model, which is much higher relative
to the cost of running the surrogate.



In [ ]:
mcmc_variable.plot_2d_marginals(
    nsamples_1d=30,
    plot_samples=[
        [post_samples, {}], [map_sample, {"c": "k", "marker": "X", "s": 100}]])
if savefig:
    plt.savefig("posterior-samples.pdf", bbox_inches="tight")

In the Bayesian inference above we used a fixed number of observations
at randomly chosen spatial locations. However choosing observation locations
is usually a poor idea. Not all observations can reduce the uncertainty
in the parameters equally. Here we use Bayesian optimal experimental design
to choose the 3 best design locations from the previously observed 10 pretending
that we do not know the value of the observations.



In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
#plot a single solution to the PDE before overlaying the designs
inv_benchmark.mesh.plot(
    inv_benchmark.obs_fun._fwd_solver.solve()[:, None], 50, ax=ax)

design_candidates = inv_benchmark.mesh.mesh_pts[:, inv_benchmark.obs_indices]
oed = get_bayesian_oed_optimizer(
    "kl_params", design_candidates, inv_benchmark.obs_fun, noise_stdev,
    inv_benchmark.variable)
oed_results = []
ndesign = 3
for step in range(ndesign):
    results_step = oed.update_design()[1]
    oed_results.append(results_step)
selected_candidates = design_candidates[:, np.hstack(oed_results)]
ax.plot(design_candidates[0, :], design_candidates[1, :], "rs")
ax.plot(selected_candidates[0, :], selected_candidates[1, :], "ko")
if savefig:
    plt.savefig("oed-selected-design.pdf")

Note that typically optimal
experimental design (OED) would be used before conducting Bayesian inference.
However, because understanding of Bayesian inference is needed to understand
Bayesian OED we reversed the order. OED is much more expensive than a single
Bayesian calibration because it requires solving many calibration problems.
So typically we do not solve the calibration problems in the OED procedure
to the same degree of accuracy as a final calibration. The accuracy of the
calibrations used by OED must only be sufficient to distinguish between designs.
This accuracy is typically much lower than the accuracy required in
estimates of uncertainty in the parameters or predictions needed for decision making tasks such as risk assessment.



Here we will set up a related benchmark to the one we have been using,
which can be used to demonstrate the forward propagation of uncertainty.
This benchmark uses the steady state solution of the advection diffusion,
obtained with a constant addition of a tracer into the domain at a single
source model as initial condition. A pump at another locations is then activated
to extract the tracer from the domain. The benchmark quantity of interest
measures the change of the tracer concentration in a subomain.
The benchmark provides models of varying cost
and accuracy that use different discretizations of the spatial PDE mesh
and number of time steps which can be used with multi-fideilty methods.
To setup the benchmark use the following



In [ ]:
fwd_benchmark = setup_benchmark(
    "multi_index_advection_diffusion",
    kle_nvars=inv_benchmark.variable.num_vars(), kle_length_scale=0.5,
    time_scenario=True)
model = WorkTrackingModel(
    TimerModel(fwd_benchmark.model_ensemble), num_config_vars=1)

Here we will use Multi-fidelity statistical estimation to compute the
mean value of the QoI to account for the uncertainty in the KLE cofficients.
So first we must compute the covariance between the QoI returned by
each of our models. We use samples from the posterior. But uncommenting
the code below will use samples from the prior.



In [ ]:
npilot_samples = 10
# generate_samples = inv_benchmark.variable.rvs # for sampling from prior
generate_samples = post_samples 
cov = multifidelity.estimate_model_ensemble_covariance(
    npilot_samples, generate_samples, model,
    fwd_benchmark.model_ensemble.nmodels)[0]

By using a WorkTrackingModel we can extract the median costs
of evaluatin each model which is needed to predict the
error of the multi-fidelity estimate of the mean which we can
compare to a prediction of the single fidelity estimate that only uses
the highest fidelity model.



In [ ]:
model_costs = model.work_tracker(
    np.asarray([np.arange(fwd_benchmark.model_ensemble.nmodels)]))
# make costs in terms of fraction of cost of high-fidelity evaluation
model_costs /= model_costs[0]

Now visualize the correlation between the models and their computational
cost relative to the highest-fidelity model cost



In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(2*8, 6))
multifidelity.plot_correlation_matrix(
    multifidelity.get_correlation_from_covariance(cov), ax=axs[0])
multifidelity.plot_model_costs(model_costs, ax=axs[1])
axs[0].set_title(mathrm_label("Model covariances"))
axs[1].set_title(mathrm_label("Relative model costs"))

Now find the best multi-fidelity estimator among all avialable option
Note, he exact predicted variance will change from run to run even with the
same seed because the computational time measured will change slightly
for each run



In [ ]:
best_est, best_model_indices = (
    multifidelity.get_best_models_for_acv_estimator(
        "acvgmfb", cov, model_costs, inv_benchmark.variable, 1e2, max_nmodels=3,
        tree_depth=4))
target_cost = 1000
best_est.allocate_samples(target_cost)
print("Predicted variance", best_est.optimized_variance)

Now we can plot the relative performance of the single and multi-fidelity
estimates of the mean before requiring any additional model evaluations



In [ ]:
hf_cov, hf_cost = cov[:1, :1], model_costs[:1]
estimators = [
    multifidelity.get_estimator(
        "mc", hf_cov, hf_cost, inv_benchmark.variable),
    best_est]
target_costs = np.array([1e1, 1e2, 1e3, 1e4], dtype=int)
optimized_estimators = multifidelity.compare_estimator_variances(
    target_costs, estimators)
est_labels = mathrm_labels(["MC", "ACV"])
fig, axs = plt.subplots(1, 2, figsize=(2*8, 6))
multifidelity.plot_estimator_variances(
    optimized_estimators, est_labels, axs[0],
    ylabel=mathrm_label("Relative Estimator Variance"))
axs[0].set_xlim(target_costs.min(), target_costs.max())
nmodels = cov.shape[0]
model_labels = [
    r"$f_{%d}$" % ii for ii in np.arange(nmodels)[best_model_indices]]
multifidelity.plot_acv_sample_allocation_comparison(
    optimized_estimators[1], model_labels, axs[1])
if savefig:
    plt.savefig("acv-variance-reduction.pdf")

It is clear that the multi-fidelity estimator will be more computationally
efficient for multiple computational budgets (target costs). Once the user
is ready to actually estimate the mean QoI they can use



In [ ]:
target_cost = 10
best_est.allocate_samples(target_cost)
best_model_ensemble = ModelEnsemble(
    [fwd_benchmark.model_ensemble.functions[ii] for ii in best_model_indices])
samples, values = best_est.generate_data(best_model_ensemble)
mean = best_est(values)
print("Mean QoI", mean)

plt.show()

## References
.. [PYAPPROX2022] `Jakeman J.D., PyApprox: Enabling efficient model analysis. (2022)`_

